# Mini Project
Setelah tugas awalku selesai, aku mengecek jam dinding. Masih ada setengah jam lagi sebelum waktu istirahat. Teman-teman kerjaku yang lain juga masih sibuk. Aku pun memanfaatkan waktu senggang ini untuk berlatih lagi!

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [16]:
# Baca dataset
dataset_credit_scoring = pd.read_excel('https://storage.googleapis.com/dqlab-dataset/credit_scoring_dqlab.xlsx')
dataset_credit_scoring.head()

,kode_kontrak,pendapatan_setahun_juta,kpr_aktif,durasi_pinjaman_bulan,jumlah_tanggungan,rata_rata_overdue,risk_rating
0,AGR-000001,295,YA,48,5,61 - 90 days,4
1,AGR-000011,271,YA,36,5,61 - 90 days,4
2,AGR-000030,159,TIDAK,12,0,0 - 30 days,1
3,AGR-000043,210,YA,12,3,46 - 60 days,3
4,AGR-000049,165,TIDAK,36,0,31 - 45 days,2


In [17]:
dataset_credit_scoring['kpr_aktif'].replace(['YA', 'TIDAK'], [1, 0], inplace=True)
dataset_credit_scoring['rata_rata_overdue'].replace({'0 - 30 days':1,
                                                     '31 - 45 days':2,
                                                     '46 - 60 days':3,
                                                     '61 - 90 days':4,
                                                     '> 90 days':5}, inplace=True)

In [18]:
#Hapus kolom - kolom yang tidak digunakan sebagai variabel independen. Kolom yang bukan variabel independen adalah kode kontrak, risk rating, rata rata overdue, dan durasi_pinjaman_bulan
X = dataset_credit_scoring.drop(columns=['kode_kontrak', 'risk_rating', 'rata_rata_overdue', 'durasi_pinjaman_bulan']).values
y = dataset_credit_scoring['risk_rating'].values

In [19]:
#Gunakan pembagian data training dan testing dengan 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
#Buat base model dengan menggunakan Random Forest
rfc = RandomForestClassifier(criterion='entropy', random_state=42)
base_model = rfc.fit(X_train, y_train)

In [21]:
#Fungsi untuk mengevaluasi model berdasarkan data testing
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: %.4f degrees.' % (np.mean(errors),))
    print('Accuracy = %.2f%%.' % (accuracy,))

In [22]:
#Terapkan hyperparameter dengan nilai masing-masing parameter
#1. jumlah pohon pada random forest
n_estimators = list(np.linspace(200, 2000, num=100, dtype=np.int32))
#2. jumlah fitur yang dipertimbangkan untuk setiap pemisahan (split)
max_features = ['auto', 'sqrt']
#3. jumlah maksimum level pada setiap pohon
max_depth = list(np.linspace(10, 110, num=11, dtype=np.int32))
max_depth.append(None)
#4. jumlah minimum sample yang dibutuhkan untuk memisahkan node
min_samples_split = [2, 5, 10]
#5. jumlah minimum sample yang dibutuhkan untuk setiap leaf node
min_samples_leaf = [1, 2, 4]
#6. metode untuk memilih sampel untuk training setiap pohon
bootstrap = [True, False]

In [23]:
#membuat grid berdasarkan parameter no. 1 s/d 6
grid = {'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'bootstrap': bootstrap}

In [24]:
#Estimator untuk RandomizedSearchCV menggunakan Random Forest
rfc = RandomForestClassifier(criterion='entropy', random_state=42)

In [25]:
#Terapkan RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rfc, param_distributions=grid, n_iter=10, cv=3, verbose=0, random_state=0)
rf_random.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(criterion='entropy',
                                                    random_state=42),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 218, 236, 254,
                                                         272, 290, 309, 327,
                                                         345, 363, 381, 400,
                                                         418, 436, 454, 472,
                                                         490, 509, 527, 545,
                                                         563, 581, 600, 618,
                                                         636, 654, 672, 690,
                                                         709, 727, ...]},
                   random_state=0)

In [26]:
#Evaluasi base model
print('Base Model:')
print('-----------')
evaluate(base_model, X_test, y_test)

#Evaluasi Best Model
best_model = rf_random.best_estimator_
print('\n\nBest Hypertuned Model:')
print('----------------------')
evaluate(best_model, X_test, y_test)

Base Model:
-----------
Model Performance
Average Error: 0.2556 degrees.
Accuracy = 85.33%.


Best Hypertuned Model:
----------------------
Model Performance
Average Error: 0.2333 degrees.
Accuracy = 86.93%.
